In [3]:
#! pip install py2neo
#!pip install neo4j
from neo4j import GraphDatabase
import pandas as pd
from sodapy import Socrata
from py2neo import Graph
from neo4j import graph
from py2neo import Relationship
import os
import importlib
import dask.dataframe as dd
from dask.distributed import Client

In [365]:
client=Client()

In [366]:
client

Client Scheduler: tcp://127.0.0.1:64356 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 51.34 GB


In [ ]:
# client = Socrata("data.cms.gov", None)
#results = client.get("j75i-rw8y", limit=2000)
#tax_df = pd.DataFrame.from_records(results)
#tax_df.to_pickle('taxonomy.pkl')

In [90]:
#npi_df=pd.read_csv('npidata_pfile_20050523-20200209.csv', usecols = ['NPI', 'Entity Type Code','Provider Last Name (Legal Name)', 'Provider First Name','Provider Name Prefix Text','Provider Name Suffix Text','Provider Credential Text',
                      #        'Provider Organization Name (Legal Business Name)','Provider Other Organization Name',
                        #      'Provider First Line Business Practice Location Address', 'Provider Business Practice Location Address City Name', 'Provider Business Practice Location Address State Name','Healthcare Provider Taxonomy Code_1'])
#npi_df.to_pickle('npi.pkl')

In [74]:
#tax=pd.read_pickle('taxonomy.pkl')
#hop = pd.read_pickle('hop.pkl')
#npi = pd.read_pickle('npi.pkl')
#npi_tax_merged=pd.read_pickle('npi_tax_merged.pkl')
#clean_npi_tax=pd.read('clean_npi_tax.pkl')

In [419]:
hop = pd.read_pickle('hop.pkl')
hop_df=hop

In [400]:
hop.shape

(203330907, 6)

In [421]:
hop.shape

(203330907, 6)

In [422]:
#Filter transaction_count > 50, average_day_wait < 50
hop=hop[(hop['transaction_count'] > 50) & (hop['average_day_wait'] < 50)].reset_index(drop=True)

In [199]:
clean_npi_tax=npi_tax_merged[['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)', 'provider_taxonomy_desc']]

In [ ]:
clean_npi_tax=clean_npi_tax[clean_npi_tax.provider_taxonomy_desc.str.contains('Pharmacy')==False]

In [303]:
def internal_medcine_cleaner(raw):
    speciality=[]
    key='Internal Medicine'
    coma=','
    for i in raw.speciality.astype(str):
        if key in i and coma in i :
            speciality.append(i.split(',')[1])
        elif coma in i:
            speciality.append(i.split(',')[0])
        else:
            speciality.append(i)
    return(speciality)       

In [452]:
clean_npi_tax['speciality']=internal_medcine_cleaner(clean_npi_tax)

In [453]:
clean_npi_tax=npi_tax_merged[['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)','provider_taxonomy_desc']]
clean_npi_tax['speciality']=clean_npi_tax['provider_taxonomy_desc'].str.split('/').str[1]
#clean_npi_tax['speciality']=clean_npi_tax['speciality'].str.split(',').str[0]
clean_npi_tax['Facility']=clean_npi_tax['Provider Organization Name (Legal Business Name)']
clean_npi_tax=clean_npi_tax[['NPI', 'Entity Type Code','speciality', 'Facility']]
#clean_npi_tax.to_pickle('clean_npi_tax.pkl')

C:\Users\Lydia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Lydia\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [454]:
provider=clean_npi_tax[clean_npi_tax['Entity Type Code']==1].reset_index(drop=True)
facility=clean_npi_tax[clean_npi_tax['Entity Type Code']==2].reset_index(drop=True)
hospital=facility[facility.Facility.str.contains('HOSPITAL')==True].reset_index(drop=True)

#Filter from_npi, Entity Type = 1 

entity1filter=hop['from_npi'].isin(provider['NPI'])==False
hop=hop.drop(hop[entity1filter].index)

#Filter to_npi, Entity Type = 2 
entity2filter=hop['to_npi'].isin(npi_entitycode2['NPI'])==False
hop=hop.drop(hop[entity2filter].index)

# filter to_npi, Hospital
hospitalfilter=hop['to_npi'].isin(hospital['NPI'])==False
hop=hop.drop(hop[hospitalfilter].index)
hop=hop.reset_index(drop=True)

In [271]:
# dont forget to export clean hop.pkl

In [404]:
#hop.to_csv('filtered_hop.csv')

In [96]:
# filter Specialist (from_npi)(Orthopedic Surgeon, Cardiology, etc) 

In [274]:
#export hopping dataframe after filtering 
#hop.to_csv('filtered_hop.csv')

In [57]:
npi_taxcode=npi_df=pd.read_csv('npidata_pfile_20050523-20200209.csv', 
                   usecols = ['NPI', 'Entity Type Code','Healthcare Provider Taxonomy Code_1'])
#npi_taxcode.to_pickle('npi_taxonomy.pkl')

In [61]:
taxcode=npi_taxcode['Healthcare Provider Taxonomy Code_1'].tolist()
taxcode=set(taxcode)
#len(taxcode)

In [361]:
#only save the tax codes that are found in the npi dataset
#taxfilter=tax.provider_taxonomy_code.isin(npi_taxcode['Healthcare Provider Taxonomy Code_1'])==False
#tax=tax.drop(tax[taxfilter].index)
#tax=tax[['provider_taxonomy_code','provider_taxonomy_desc']]

In [85]:
tax['Healthcare Provider Taxonomy Code_1']=tax['provider_taxonomy_code']

In [87]:
tax=tax[['Healthcare Provider Taxonomy Code_1','provider_taxonomy_desc']]

In [91]:
npi_tax_merged=pd.merge(npi_df,tax)

In [455]:
from_npi_filter=clean_npi_tax['NPI'].isin(hop.from_npi)==False
from_specialists_df=clean_npi_tax.drop(clean_npi_tax[from_npi_filter].index)
from_specialists_df['from_npi']=from_specialists_df['NPI']
from_specialists_df['from_speciality']=from_specialists_df['speciality']
from_specialists_df=from_specialists_df[['from_npi', 'from_speciality']].reset_index(drop=True)

In [447]:
to_npi_filter=clean_npi_tax['NPI'].isin(hop.to_npi)==False
to_hospitals_df=clean_npi_tax.drop(clean_npi_tax[to_npi_filter].index)
to_hospitals_df['to_npi']=to_hospitals_df['NPI']
to_hospitals_df['to_Facility']=to_hospitals_df['Facility']
to_hospitals_df=to_hospitals_df[['to_npi', 'to_Facility']].reset_index(drop=True)

In [461]:
hop4neo=pd.merge(hop,from_specialists_df)
hop4neo=pd.merge(hop4neo,to_hospitals_df)

In [463]:
#Within a local CBSA, show which hospitals specialists typically refer patients to
hop4neo.head(1)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,from_speciality,to_Facility
0,1013177500,1003014952,53,54,45.333,48.711,Emergency Medicine,SONORA COMMUNITY HOSPITAL
1,1043345366,1003014952,81,99,44.869,52.187,Ophthalmology,SONORA COMMUNITY HOSPITAL
2,1003245895,1003014952,23,217,0.000,0.000,"Psychologist, Clinical",SONORA COMMUNITY HOSPITAL
3,1003245895,1003014952,23,217,0.000,0.000,"Psychologist, Clinical",SONORA COMMUNITY HOSPITAL
4,1083014559,1003014952,52,54,38.556,52.071,"Nurse Practitioner, Family",SONORA COMMUNITY HOSPITAL


In [471]:
hop_ready4neo=hop4neo[['from_speciality', 'to_Facility','patient_count', 'transaction_count','average_day_wait', 'std_day_wait']]

In [473]:
hop_ready4neo.to_csv('hop_ready4neo.csv')

In [479]:
small_hop_ready4neo=pd.read_csv('hop_ready4neo.csv', nrows=1000)

In [480]:
small_hop_ready4neo.to_csv('small_hop_ready4neo.csv')

In [481]:
hop_ready4neo.head()

,from_speciality,to_Facility,patient_count,transaction_count,average_day_wait,std_day_wait
0,Emergency Medicine,SONORA COMMUNITY HOSPITAL,53,54,45.333,48.711
1,Ophthalmology,SONORA COMMUNITY HOSPITAL,81,99,44.869,52.187
2,"Psychologist, Clinical",SONORA COMMUNITY HOSPITAL,23,217,0.000,0.000
3,"Psychologist, Clinical",SONORA COMMUNITY HOSPITAL,23,217,0.000,0.000
4,"Nurse Practitioner, Family",SONORA COMMUNITY HOSPITAL,52,54,38.556,52.071


In [98]:
nodes = pd.DataFrame(nodedata)
edges = pd.DataFrame(edgedata)
nodePath = 'nodes.csv'
edgePath = 'edges.csv'

In [ ]:
serverURL = 'localhost:7474'
dbURL = 'http://localhost:7687/db/data'
account = "neo4j"
password = "****"

In [ ]:
nodes.to_csv(nodePath, index=False)
edges.to_csv(edgePath, index=False)

In [18]:
graph = Graph(dbURL)

In [ ]:
def path2url(path):
    return ('file:', u(path))


In [ ]:
#cypher params
nodename = "Provider"
edgename = "patient_count"
source_name = "from_npi"
dest_name = "to_npi"

In [ ]:
graph.cypher.execute("LOAD CSV WITH HEADERS FROM '%s' AS csvLine " % (path2url(nodePath)) +
                     " CREATE (p:"+nodename+" { id: toInt(csvLine.subid), name: csvLine.name, age: toInt(csvLine.age), 
                     fdg: toFloat(csvLine.fdg) })")


In [ ]:
#import edges
tx = graph.cypher.begin()
statement = "USING PERIODIC COMMIT 500" \
            + " LOAD CSV WITH HEADERS FROM '" + path2url(edgePath) + "' AS csvLine" \
            + " MATCH (person:"+source_name+" { id: toInt(csvLine.source)}),(person1:"+dest_name+" { id: toInt(csvLine.dest)})" \
            + " CREATE (person)-[:"+edgename+" { rating: toInt(csvLine.rating), weight: toInt(csvLine.weight) }]->(person1)"
tx.append(statement)
tx.commit()

In [ ]:

#NODE BY NODE IMPORT
for index, row in nodes.iterrows():
    node = Node(nodename, id=int(row['subid']), age = row['age'], fdg = row['fdg'], name = row['name'])
    graph.create(node)


In [ ]:
for index, row in hop.iterrows():
    source_node = graph.find_one(nodename, property_key='id', property_value=int(row['source']))
    dest_node = graph.find_one(nodename, property_key='id', property_value = int(row['dest']))
    relation = Relationship(source_node, edgename, dest_node, rating = int(row['rating']))
    graph.create(relation)


In [ ]:
CODE BACKUP
# Load CSV
LOAD CSV WITH HEADERS FROM 'file:///small_hopping.csv' AS line
CREATE (:Provider { npi: line.from_npi}) -[:Referred {patient_count: line.patient_count}]-> (:Provider {npi: line.to_npi})

#LOAD WITH PROPER EDGES/NODES
LOAD CSV WITH HEADERS FROM 'file:///small_hopping.csv' AS line
MERGE (f:Provider { npi: line.from_npi} )
MERGE (t:Provider { npi: line.to_npi} )
MERGE (f)-[:Referred {patient_count: line.patient_count}]->(t)

#Delete all nodes and edges
MATCH (p:Provider) DETACH DELETE p




In [ ]:
LOAD CSV WITH HEADERS FROM 'file:///small_hop_ready4neo.csv' AS row 
MERGE (p1:Provider {npi:row.from_speciality}) 
MERGE (p2:Provider {npi:row.to_Facility}) 
MERGE (p1)-[:REFERRED_TO {patient_count:row.patient_count, transaction_count:row.transaction_count, 
average_day_wait:row.average_day_wait}]->(p2);
MATCH (p1:Provider)-[c:REFERRED_TO]->(p2:Provider) return p1,c,p2